In [2]:
# import
import re
import csv
import numpy as np
import pandas as pd
import MeCab
from ekonlpy.sentiment import MPCK
from ekonlpy.tag import Mecab

In [35]:
bond_label = pd.read_csv('pre_processing_bond_final.csv')
bond_label

bond_label.dropna(inplace=True)
bond_label.reset_index(inplace=True, drop=True)
bond_label

,date,title,content,tokens,ngrams,call_rate,call_label
0,2008-04-07,돌아온 박스권,ㅎㅎㅎ2008_04 월간채권투자채 권 시 장 전 망금 융 시 장 차 트 북對應과 ...,"['가/VV', '채권투자/NNG', '채권시장/NNG', '전망/NNG', '금융...","['미/NNG;국채/NNG;금리/NNG;안전자산/NNG;선호/NNG', '경기/NN...",5.04,increase
1,2008-04-14,총재의 변신은 무죄다?!,Microsoft Word - 0407.docFixed Income Weekly 2...,"['돌아온/VV', '박스권/NNG', '하단/NNG', '상단/NNG', '테스트...","['글로벌/NNG;신용경색/NNG;완화/NNG', '단기/NNG;인플레이션/NNG;...",5.00,increase
2,2008-04-21,항상 열려는 있는 가능성,Microsoft Word - Fixedincome0414.doc 4월 금통위...,"['믹스/NNG', '합의/NNG', '반영/NNG', '예상/NNG', '경제/N...","['예상/NNG;경제/NNG;성장률/NNG;하향/NNG', '금리/NNG;인하/NN...",4.97,decrease
3,2008-04-28,FOMC 이후 원자재 가격 상승세 둔화 여부에 관심,Microsoft Word - 920_0421_Fixed Income Weekly....,"['경기/NNG', '전망/NNG', '비교적/VAX', '분명/MAG', '인플레...","['미/NNG;국채/NNG;금리/NNG;안전자산/NNG;선호/NNG', '금리/NN...",4.89,decrease
4,2008-05-04,정부의 금리인하 요구에 한은의 부응 제스쳐 나타나며..,Microsoft Word - FixedIncome_0428.doc FOMC ...,"['fed/NNG', '원자재/NNG', '가격/NNG', '상승/NNG', '둔화...","['fed/NNG;원자재/NNG;가격/NNG;상승/NNG', '원자재/NNG;가격/...",4.96,decrease
...,...,...,...,...,...,...,...
1652,2017-12-15,12월 FOMC 함의와 2018년 연준구성 변화,Microsoft Word - 2017121407564004.docx 채권분석 2...,"['채권분석/NNG', '채권/NNG', 'fed/NNG', '통화정책/NNG', ...","['fed/NNG;금리/NNG;인상/NNG;가능성/NNG;작/VA', '금리/NNG...",1.51,increase
1653,2017-12-18,속도 조절 중,12월 FOMC: 성장과 물가의 Dilemma • 12월 FO...,"['fed/NNG', '성장/NNG', '물가/NNG', 'fed/NNG', 'ff...","['fed/NNG;ff/NNG;목표/NNG;금리/NNG;인상/NNG', '금리/NN...",1.50,increase
1654,2017-12-20,확신 없는 인상의견도 있었음이 확인,(Microsoft Word - \307\366\264\353\302\367_Cre...,"['속도조절/NNG', '중/NNG', '발행시장/NNG', '공모/NNG', '회...","['크레딧/NNG;시장/NNG;스프레드/NNG;축소/NNG', '전망/NNG;상향/...",1.51,increase
1655,2017-12-26,11월 채권시장 전망,2013년 0월 0일 2금융권 안심전환대출 수급영향 점검 Inve...,"['금융/NNG', '수급/NNG', '영향/NNG', '점검/NNG', '채권/N...","['국고채/NNG;금리/NNG;급등/NNG', '원금/NNG;상환/NNG;부담/NN...",1.52,increase


In [57]:
bond_test = pd.read_csv('bond_test.csv')
bond_test_tokens = bond_test['token_list'][0]

bond_test_tokens = bond_test_tokens.replace('[',"")
bond_test_tokens = bond_test_tokens.replace(']',"")
bond_test_tokens = bond_test_tokens.replace('\'',"")
bond_test_tokens= bond_test_tokens.split(', ')
print(bond_test_tokens[0])

가/VV


In [64]:
test = pd.DataFrame(bond_label['tokens'][:10])
test

,tokens
0,"['가/VV', '채권투자/NNG', '채권시장/NNG', '전망/NNG', '금융..."
1,"['돌아온/VV', '박스권/NNG', '하단/NNG', '상단/NNG', '테스트..."
2,"['믹스/NNG', '합의/NNG', '반영/NNG', '예상/NNG', '경제/N..."
3,"['경기/NNG', '전망/NNG', '비교적/VAX', '분명/MAG', '인플레..."
4,"['fed/NNG', '원자재/NNG', '가격/NNG', '상승/NNG', '둔화..."
5,"['가/VV', '채권투자/NNG', '채권시장/NNG', '전망/NNG', '금융..."
6,"['레인지/NNG', '상단부/NNG', '기회/NNG', '다시/MAG', '만들..."
7,"['채권/NNG', '시황/NNG', '만기/NNG', '금리/NNG', '상승/N..."
8,"['채권/NNG', '시황/NNG', '만기/NNG', '금리/NNG', '상승/N..."
9,"['채권/NNG', '시황/NNG', '만기/NNG', '금리/NNG', '하락/N..."


In [68]:
for i, tokens in enumerate(test):
    tokens = tokens.replace('[',"")
    tokens = tokens.replace(']',"")
    tokens = tokens.replace('\'',"")
    tokens = tokens.split(', ')
    t_list = []
    for j in tokens:
        if j in bond_test_tokens:
            t_list.append(j)
    test['tokens'][i] = t_list
test

,tokens
0,[]
1,"['돌아온/VV', '박스권/NNG', '하단/NNG', '상단/NNG', '테스트..."
2,"['믹스/NNG', '합의/NNG', '반영/NNG', '예상/NNG', '경제/N..."
3,"['경기/NNG', '전망/NNG', '비교적/VAX', '분명/MAG', '인플레..."
4,"['fed/NNG', '원자재/NNG', '가격/NNG', '상승/NNG', '둔화..."
5,"['가/VV', '채권투자/NNG', '채권시장/NNG', '전망/NNG', '금융..."
6,"['레인지/NNG', '상단부/NNG', '기회/NNG', '다시/MAG', '만들..."
7,"['채권/NNG', '시황/NNG', '만기/NNG', '금리/NNG', '상승/N..."
8,"['채권/NNG', '시황/NNG', '만기/NNG', '금리/NNG', '상승/N..."
9,"['채권/NNG', '시황/NNG', '만기/NNG', '금리/NNG', '하락/N..."
